In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [6]:
## Data Ingestions Step
df = pd.read_csv("finalTrain.csv")

In [7]:
df_copy = df.copy()

In [8]:
df_copy = df_copy.drop(labels=['ID', 'Delivery_person_ID', 'Order_Date', 'Time_Orderd', 'Time_Order_picked'],axis=1)
df_copy.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [9]:
R = 6371  ##The earth's radius (in km)

def deg_to_rad(degrees):
    return degrees * (np.pi/180)

## The haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a1 = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1))
    a2 = np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    a = a1 * a2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# Create distance column & calculate the distance
df_copy['distance'] = np.nan

for i in range(len(df_copy)):
  df_copy.loc[i, 'distance'] = distcalculate(df_copy.loc[i, 'Restaurant_latitude'], 
                                          df_copy.loc[i, 'Restaurant_longitude'], 
                                          df_copy.loc[i, 'Delivery_location_latitude'], 
                                          df_copy.loc[i, 'Delivery_location_longitude'])

In [10]:
df_copy = df_copy.drop(labels=['Restaurant_latitude',	'Restaurant_longitude',	'Delivery_location_latitude', 'Delivery_location_longitude'],axis=1)
df_copy.head()

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),distance
0,36.0,4.2,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,6.716044
1,21.0,4.7,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,4.379914
2,23.0,4.7,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,9.484418
3,34.0,4.3,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,1.908058
4,24.0,4.7,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,12.933369


These graphs shows that the weather condition, type of vehicle, the type of order, vehicle condition do not significantly affect delivery time

In [11]:
df_copy = df_copy.drop(labels=['Type_of_order'],axis=1)
df_copy.head()

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),distance
0,36.0,4.2,Fog,Jam,2,motorcycle,3.0,No,Metropolitian,46,6.716044
1,21.0,4.7,Stormy,High,1,motorcycle,1.0,No,Metropolitian,23,4.379914
2,23.0,4.7,Sandstorms,Medium,1,scooter,1.0,No,Metropolitian,21,9.484418
3,34.0,4.3,Sandstorms,Low,0,motorcycle,0.0,No,Metropolitian,20,1.908058
4,24.0,4.7,Fog,Jam,1,scooter,1.0,No,Metropolitian,41,12.933369


In [12]:
## Independent and dependent feature
X = df_copy.drop(labels=['Time_taken (min)'],axis=1)
Y = df_copy[['Time_taken (min)']]

In [13]:
# Define which columns should be ordinal-encoded and which should be scale
categorical_cols = X.select_dtypes(include='object') .columns
numerical_cols = X.select_dtypes(exclude='object').columns
print("categorical columns: ", categorical_cols)
print("numerical columns: ",numerical_cols)

categorical columns:  Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_vehicle',
       'Festival', 'City'],
      dtype='object')
numerical columns:  Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition',
       'multiple_deliveries', 'distance'],
      dtype='object')


In [165]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_vehicle',
       'Festival', 'City'],
      dtype='object')

In [166]:
for i in categorical_cols:
    print('\n')
    print(df_copy[i].unique())



['Fog' 'Stormy' 'Sandstorms' 'Windy' 'Cloudy' 'Sunny' nan]


['Jam' 'High' 'Medium' 'Low' nan]


['motorcycle' 'scooter' 'electric_scooter' 'bicycle']


['No' 'Yes' nan]


['Metropolitian' 'Urban' 'Semi-Urban' nan]


In [167]:
# Define the custom ranking for each ordinal variable
Weather_condition_categories = ['Fog', 'Stormy', 'Sandstorms', 'Windy', 'Cloudy', 'Sunny']
Road_traffic_density_categories = ['Jam', 'High', 'Medium', 'Low']
Type_of_vehicle_categories = ['motorcycle', 'scooter', 'electric_scooter', 'bicycle']
Festival_categories = ['No', 'Yes']
City_categories = ['Metropolitian', 'Urban', 'Semi-Urban']

In [170]:
from sklearn.impute import SimpleImputer  # Handling missing values
from sklearn.preprocessing import StandardScaler  # Handling feature  scaling
from sklearn.preprocessing import OrdinalEncoder  # ordinal encodind
## Pipelines 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [171]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[Weather_condition_categories,Road_traffic_density_categories, Type_of_vehicle_categories, Festival_categories, City_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [172]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())
X_train.head()

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__distance,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City
0,1.124025,-1.623440,-1.223528,-1.318236,-0.058522,-0.261197,-1.308057,-0.777630,-0.142953,-0.532682
1,-0.106505,-3.143059,-1.223528,3.943714,-0.059550,1.482131,-1.308057,-0.777630,6.995304,1.805257
2,0.245075,0.504025,1.160323,0.435747,-0.063728,0.319912,1.093916,0.753919,-0.142953,-0.532682
3,0.596655,-1.319517,1.160323,0.435747,-0.051777,0.319912,-1.308057,0.753919,-0.142953,-0.532682
4,0.772445,-1.623440,-1.223528,0.435747,-0.063617,0.319912,1.093916,-0.777630,-0.142953,1.805257


In [174]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [175]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [176]:
regression.coef_

array([[ 2.26479547, -2.33885315, -1.77939703,  2.06940285,  0.19310146,
        -1.16195449, -3.20472581, -0.0213674 ,  1.61039806, -0.75652328]])

In [177]:
regression.intercept_

array([26.31769462])

In [178]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [179]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 6.681032631391504
MAE: 5.271442963891952
R2 score 48.39442696549199


Lasso
Model Training Performance
RMSE: 7.081947624124815
MAE: 5.638425369306291
R2 score 42.015109674383055


Ridge
Model Training Performance
RMSE: 6.681030241866386
MAE: 5.271441236746989
R2 score 48.39446387978285


Elasticnet
Model Training Performance
RMSE: 7.138249710610972
MAE: 5.698084806403554
R2 score 41.089475177376045




In [180]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [181]:
features = np.array([[1.127187,	-1.616514,	-1.318236,	-0.058522,	-1.308057,	-0.142953,	-0.532682]])
print("Delivery Time Prediction in Minutes = ", model.predict(features))

d:\projects\fooddeliverytime\venv\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(


ValueError: X has 7 features, but ElasticNet is expecting 10 features as input.